In [1]:
from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import seaborn as sns

import shared_utils
import utils
import os

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt

import gcsfs
fs = gcsfs.GCSFileSystem()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [ ]:
endpoint_delay_all = pd.DataFrame()
for operator_parquet in os.listdir('./endpoint_delay_views/'):
    if operator_parquet == '.ipynb_checkpoints':
        continue
    calitp_itp_id = operator_parquet.split('_')[0]
    df = gpd.read_parquet(f'./endpoint_delay_views/{operator_parquet}')
    df['calitp_itp_id'] = int(calitp_itp_id)
    endpoint_delay_all = pd.concat([endpoint_delay_all, df])

In [ ]:
endpoint_delay_all >> head(5)

In [ ]:
endpoint_delay_all.delay_seconds.sum() / 60 ** 2 ## ~3340 hours of delay on a weekday, statewide

In [2]:
gdf = shared_utils.utils.download_geoparquet(utils.BUS_GCS_FILE_PATH, 
                                             "parallel_or_intersecting")

In [3]:
parallel_routes = (gdf
                   >> filter(_.pct_route >= .3)
                   >> select(_.calitp_itp_id == _.itp_id, _.route_id)
                   >> distinct(_.calitp_itp_id, _.route_id)
                  )

In [ ]:
parallel_delays = (endpoint_delay_all
                   >> inner_join(_, parallel_routes, on = ['calitp_itp_id', 'route_id'])
                   >> distinct(_.calitp_itp_id, _.trip_id, _keep_all=True)
                   ## ensure we don't duplicate delays...
                   ## guessing these come partially from multiple url numbers/aggregation in RT tools?
                  )

In [ ]:
parallel_delays.delay_seconds.sum() / 60 ** 2

In [ ]:
parallel_delays.route_id.isin(gdf.route_id).all() ## looks good

#### what about parallel routes without RT data?

In [ ]:
parallel_routes

In [ ]:
parallel_delays >> distinct(_.calitp_itp_id, _.route_id)

In [4]:
class ThinMapper(rt.OperatorDayAnalysis):
    
    def __init__(self, segment_view):
        self.stop_segment_speed_view = segment_view
        self.calitp_agency_name = 'Statewide'
        self.filter_formatted = 'PM Peak'

In [5]:
segment_all = pd.DataFrame()
for operator_parquet in os.listdir('./segment_speed_views/'):
    if operator_parquet == '.ipynb_checkpoints':
        continue
    calitp_itp_id = operator_parquet.split('_')[0]
    df = gpd.read_parquet(f'./segment_speed_views/{operator_parquet}')
    df['calitp_itp_id'] = int(calitp_itp_id)
    segment_all = pd.concat([segment_all, df])

In [6]:
segment_all = segment_all >> distinct(_.shape_id, _.stop_sequence, _keep_all=True) ## essential here for reasonable map size!

In [7]:
segment_filtered = segment_all >> inner_join(_, parallel_routes >> select(_.calitp_itp_id, _.route_id), on = ['calitp_itp_id', 'route_id'])

In [8]:
segment_filtered.to_parquet('segment_all.parquet')

/tmp/ipykernel_925/1034411430.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


In [9]:
mapper = ThinMapper(segment_filtered)

In [10]:
m = mapper._show_speed_map('low_speeds', utils.ZERO_THIRTY_COLORSCALE, [1300, 700])

TopologyException: assigned depths do not match at -166523.38192423596 -35415.952393142536
TopologyException: assigned depths do not match at -166523.38192423596 -35415.952393142536
TopologyException: assigned depths do not match at -128381.9227042797 -165982.48672357475
TopologyException: assigned depths do not match at -117953.55795454726 1648.8320321653728
TopologyException: assigned depths do not match at -217628.67881293956 -35629.731197568181
TopologyException: assigned depths do not match at -217628.67881293956 -35629.731197568181
TopologyException: assigned depths do not match at -224574.96029423364 -10595.528804516269
TopologyException: assigned depths do not match at -231881.71140996384 -9874.5257799801257
TopologyException: assigned depths do not match at -231947.21270706289 -9882.1030193925762
TopologyException: assigned depths do not match at -206250.92608763179 -51291.043813443037
TopologyException: assigned depths do not match at -128570.88196397232 -166062.66690813107
T

In [11]:
m.save('statewide_shn_corridors.html')